# SchemaGen Test run

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AseiSugiyama/TFXTestRun/blob/master/notebooks/SchemaGenTestRun.ipynb)

## Set up

TFX requires apache-airflow and docker SDK.

In [ ]:
!pip install 'apache-airflow[gcp]' docker tfx

In this notebook, we use TFX version 0.13.0

In [ ]:
import tfx
tfx.version.__version__

TFX requires TensorFlow >= 1.13.1

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

TFX supports Python 3.5 from version 0.13.0

In [ ]:
import sys
sys.version

## Download sample data

In [ ]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration.tfx_runner import TfxRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

## configs

In [ ]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [ ]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

# Create outputs
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'train/')

eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'eval/')

output_dict = {'examples': Channel(
    type_name='ExamplesPath',
    static_artifact_collection=[train_examples, eval_examples])}

example_outputs = ComponentOutputs(output_dict)

example_gen = CsvExampleGen(
    input_base=examples, # A Channel of 'ExternalPath' type, it contains path of data source.
    output_config=output_config,  # An example_gen_pb2.Output instance, it contains train-eval split ratio.
    outputs=example_outputs # dict from name to output channel, it will be stored example_gen.outputs
)

## Create StatisticsGen

In [ ]:
# Create outputs
train_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='train')
train_statistics.uri = os.path.join(_data_root, 'train/stats/')

eval_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='eval')
eval_statistics.uri = os.path.join(_data_root, 'eval/stats/')

output_dict = {'output': Channel(
    type_name='ExampleStatisticsPath',
    static_artifact_collection=[train_statistics, eval_statistics])}

statistics_outputs = ComponentOutputs(output_dict)

statistics_gen = StatisticsGen(
    input_data=example_gen.outputs.examples, # A Channel of 'ExamplesPath' type, it is equal to example_outputs
    name='Statistics Generator', # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=statistics_outputs # dict from name to output channel, it will be stored statistics_gen.outputs
)

## Create SchemaGen

In [ ]:
# Create outputs
train_schema_path = types.TfxType(type_name='SchemaPath', split='train')
train_schema_path.uri = os.path.join(_data_root, 'train/schema/')

schema_outputs = ComponentOutputs({
    'output':Channel(
        type_name='SchemaPath',
        # SchemaGen.executor can handle just one SchemaPath.
        # Two or more SchemaPaths will cause ValueError
        # such as "ValueError: expected list length of one but got 2".
        static_artifact_collection=[train_schema_path] 
    )
})

schema_gen = SchemaGen(
    stats=statistics_gen.outputs.output, # A Channel of 'ExampleStatisticsPath' type, it is equal to statistics_outputs
    name='Schema Generator',  # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=schema_outputs # dict from name to output channel, it will be stored schema_gen.outputs
)

## Create pipeline

In [ ]:
pipeline = Pipeline(
    pipeline_name="TFX Pipeline",
    pipeline_root=_pipeline_root,
    components=[example_gen, statistics_gen, schema_gen]
)

## Execute

In [ ]:
class DirectRunner(TfxRunner):
    """Tfx runner on local"""
    
    def __init__(self, config=None):
        self._config = config or {}
    
    def run(self, pipeline):
        for component in pipeline.components:
            self._execute_component(component)
            
        return pipeline
            
    def _execute_component(self, component):
        input_dict = {key:value.get() for key, value in component.input_dict.items()}
        output_dict = {key: value.get() for key, value in component.outputs.get_all().items()}
        exec_properties = component.exec_properties
        executor = component.executor()
        executor.Do(input_dict, output_dict, exec_properties)

In [ ]:
pipeline = DirectRunner().run(pipeline)

## Check Result

In [ ]:
!ls -Rlhs /root/taxi/data/simple/

In [ ]:
!cat /root/taxi/data/simple/train/schema/schema.pbtxt

`schema.pbtxt` is only simple text file, we are going to;

1. get the path of `schema.pbtxt`from `schema_gen`
2. parse `schema.pbtxt` into schema (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)


In [ ]:
# 1. get the path of `schema.pbtxt`
def get_schema_directory(schema_gen):
    output_dict = {key: value.get() for key, value in schema_gen.outputs.get_all().items()}
    input_dict = {key:value.get() for key, value in schema_gen.input_dict.items()}
    split_to_instance = {x.split: x for x in input_dict['stats']}
    directory = types.get_split_uri(output_dict['output'], 'train')
    return directory

schema_directory = get_schema_directory(schema_gen)

In [ ]:
# 2. parse schema.pbtxt

from pathlib import Path
from google.protobuf.text_format import Parse
from google.protobuf.message import Message
from tensorflow_metadata.proto.v0 import schema_pb2

def parse_schema_proto_string(schema_directory):
    path = Path(schema_directory)
    for filepath in path.glob('*'):
        with open(str(filepath), 'r') as file: # since we are using python 3.5, not 3.6+
            schema_proto_string = file.read()
            schema = schema_pb2.Schema()
            text_format.Parse(schema_proto_string, schema)
            return schema

schema = parse_schema_proto_string(schema_directory)

In [ ]:
# 3. visualize schema with tfdv

import tensorflow_data_validation as tfdv
tfdv.display_schema(schema)